w_02 | Descargar archivos programáticamente con `requests`
====

Lo primero que vamos a hacer es importar los paquetes necesarios. El foco para esta parte son `requests` y `re`, pero voy a explicar los usos de los restantes también.

A esta altura debería estar razonablemente claro para qué usamos `requests`. Para el resto:

* `re` es el paquete standar de RegEx de Python, que permite trabajar con expresiones regulares de texto.
* `pathlib` es un paquete estándar para manejar paths de manera de que el código funcione en cualquier sistema operativo, superando el drama de forward y backward slashes dependiendo de la plataforma. Recomiendo siempre usarlo para evitar dolores de cabeza.

In [ ]:
import re
from pathlib import Path

import requests

Lo que vamos a hacer es descargar los datos de proyectos recomendados por la COMAP para que su inversión sea promovida por el Poder Ejecutivo en el marco de la Ley de Inversiones. Estos datos están contenidos en PDFs por año en la web de la COMAP.

Empecemos por 2020. Para ello definimos la URL relevante y vamos a hacerle un `get()` para acceder.

In [ ]:
url_2020 = "https://www.gub.uy/ministerio-economia-finanzas/sites/ministerio-economia-finanzas/files/2021-01/comap_setiembre-2020.pdf"
r_2020 = requests.get(url_2020)
r_2020.status_code

El status_code 200 indica que se pudo acceder de manera exitosa.

Recordar que `get()` devuelve un `Response` object. En este caso, este objeto es un PDF. Para descargarlo vamos a "escribir" su contenido en un archivo, pero antes tenemos que crear una carpeta en donde guardar el PDF.

El objeto `Path()` es el constructor de rutas de `pathlib`. La gracia es que no hay necesidad de lidiar con idiosincracias del sistema operativo y simplemente podemos definir la ruta con argumentos del objeto. Cada argumento es un nivel adicional de carpetas; partimos de la raíz y vamos agregando niveles. Cabe destacar que `Path()` no crea la ruta, sino que simplemente es una referencia a ella.

Para crearla corremos el método `mkdir()` sobre el `Path` creado.

In [ ]:
download_dir = Path("02_w", "data", "comap_pdfs")
download_dir.mkdir(parents=True, exist_ok=True)

Ya con la carpeta creada podemos escribir los contenidos de la respuesta en un archivo, valiéndonos de un context manager.

Los context managers de Python se encargan de abrir y cerrar archivos una vez que ya fueron utilizados.

In [ ]:
path_2020 = Path(download_dir, "2020-set.pdf")
with open(path_2020, "wb") as f:
    f.write(r_2020.content)

Eso es el proceso para un PDF. Pero hay varios, uno por cada año, con URLs no uniformes.

Para encontrar todas las URLs relevantes vamos a usar un RegEx simple: en la página de descargas buscamos todas las instancias de texto que empiece con "https" y termine con "pdf", sin importar qué haya en el medio.

Como resultado obtenemos una lista con todos los matches.

In [ ]:
url = "https://www.gub.uy/ministerio-economia-finanzas/datos-y-estadisticas/estadisticas/estadisticas-proyectos-enviados-poder-ejecutivo-recomendacion-aprobacion-comap"
r = requests.get(url)
pdfs = re.findall("http[A-z0-9\.\/:-]+pdf", r.text)
pdfs

Como en la lista de PDFs aparece 2020, que ya descargamos, primero vamos a excluirlo. Siguiente, vamos a crear una lista de nombres para uniformizarlos (2019.pdf, 2018.pdf, ..., 2009.pdf).

Luego repetimos el paso para escribir el contenido en un for loop, pero tenemos que
* Iterar sobre la lista de PDFs y los nombres en paralelo, para lo cual usamos `zip()`.
* Hacer un request en cada iteración.
* Crear un nuevo path en cada iteración, incorporando el nombre correspondiente.
* Finalmente escribir el contenido en el path correspondiente.

In [ ]:
pdfs = pdfs[1:]
filenames = [f"{x}.pdf" for x in range(2009, 2020)][::-1]

for pdf, filename in zip(pdfs, filenames):
    r_pdf = requests.get(pdf)
    path_pdf = Path(download_dir, filename)
    with open(path_pdf, "wb") as f:
        f.write(r_pdf.content)

Ese es el proceso usual para descargar archivos usando `requests`. En este caso eran PDFs, pero puede ser cualquier tipo de archivo.